In [1]:
import os

In [2]:
import shutil

In [3]:
old_train_data_dir_path=os.path.join("Data","cifar10","train")

In [4]:
classes_config={
    "class1":["airplane","automobile","ship","truck"],
    "class2":["cat","deer","dog","horse"],
    "class3":["bird","frog"]
}

In [5]:
new_train_data_dir_path=os.path.join("Data","cifar_train_data_new")
os.makedirs(new_train_data_dir_path,exist_ok=True)

In [6]:
class1_dir=os.path.join(new_train_data_dir_path,"class1")
class2_dir=os.path.join(new_train_data_dir_path,"class2")
class3_dir=os.path.join(new_train_data_dir_path,"class3")

In [7]:
os.makedirs(class1_dir,exist_ok=True)
os.makedirs(class2_dir,exist_ok=True)
os.makedirs(class3_dir,exist_ok=True)

In [8]:
for img_type in os.listdir(old_train_data_dir_path):
   
    old_dir=os.path.join(old_train_data_dir_path,img_type)
    
    if img_type in classes_config["class1"]:
        new_dir=os.path.join(class1_dir,img_type)
        shutil.copytree(old_dir,new_dir,dirs_exist_ok=True)
        
    elif img_type in classes_config["class2"]:
        new_dir=os.path.join(class2_dir,img_type)
        shutil.copytree(old_dir,new_dir,dirs_exist_ok=True)
        
    elif img_type in classes_config["class3"]:
        new_dir=os.path.join(class3_dir,img_type)
        shutil.copytree(old_dir,new_dir,dirs_exist_ok=True)
        

In [9]:
new_train_data_dir_path

'Data/cifar_train_data_new'

In [10]:
os.listdir(new_train_data_dir_path)

['class3', 'class1', 'class2']

In [11]:
def get_filepaths(root_dir):
    dir_level1=[os.path.join(root_dir,path) for path in os.listdir(root_dir)]
    dir_level2=[os.path.join(dir1,dir2) for dir1 in dir_level1 for dir2 in os.listdir(dir1)]
    filepaths=[os.path.join(dir2,path) for dir2 in dir_level2 for path in os.listdir(dir2)]
    return filepaths

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer

2025-02-23 04:06:34.786914: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-23 04:06:34.796911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740263794.812204    6543 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740263794.816481    6543 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-23 04:06:34.829724: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [13]:
class_tokenizer=Tokenizer()
class_tokenizer.fit_on_texts(list(classes_config.keys()))

In [14]:
class_tokenizer.word_index

{'class1': 1, 'class2': 2, 'class3': 3}

In [15]:
class1_tokenizer=Tokenizer()
class1_tokenizer.fit_on_texts(list(classes_config["class1"]))

In [16]:
class1_tokenizer.word_index

{'airplane': 1, 'automobile': 2, 'ship': 3, 'truck': 4}

In [17]:
class2_tokenizer=Tokenizer()
class2_tokenizer.fit_on_texts(list(classes_config["class2"]))
class2_tokenizer.word_index

{'cat': 1, 'deer': 2, 'dog': 3, 'horse': 4}

In [18]:
class3_tokenizer=Tokenizer()
class3_tokenizer.fit_on_texts(list(classes_config["class3"]))
class3_tokenizer.word_index

{'bird': 1, 'frog': 2}

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
max_class_number=4

In [21]:
label_tokenizers={
    "class_tokenizer":class_tokenizer.word_index,
    "class1_tokenizer":class1_tokenizer.word_index,
    "class2_tokenizer":class2_tokenizer.word_index,
    "class3_tokenizer":class3_tokenizer.word_index,
}

In [22]:
from tensorflow.keras.utils import to_categorical

In [23]:
to_categorical(2,3)

array([0., 0., 1.])

In [24]:
max_subclass_in_class=4

In [25]:
img_size=[256,256]

In [26]:
filepaths=get_filepaths(new_train_data_dir_path)

In [27]:
def preprocess_image(filepath,max_num_subclass=4,label_tokenizers=label_tokenizers):
     #Load Image
    img=tf.io.read_file(filepath)
    img=tf.io.decode_jpeg(img,channels=3)
    img=tf.image.resize(img,img_size)
    img/=255.0
   
    
    # Labels:
    class_tokenizer=label_tokenizers["class_tokenizer"]
    class_name=filepath.split("/")[-3]
    sub_class=filepath.split("/")[-2]

    # Label1:
    label1=class_tokenizer[class_name]-1
    label1=to_categorical(label1,len(class_tokenizer))
    label1=tf.cast(label1,dtype=tf.float32)
    
    # Label2
    sub_class_tokenizer=label_tokenizers[f"{class_name}_tokenizer"]
    label2=sub_class_tokenizer[sub_class]-1
    label2=to_categorical(label2,len(sub_class_tokenizer))
    label2=pad_sequences([label2],maxlen=max_num_subclass,padding="post")[0]
    label2=tf.cast(label2,dtype=tf.float32)

    image_bytes=tf.io.serialize_tensor(img).numpy()
    label1_bytes=tf.io.serialize_tensor(label1).numpy()
    label2_bytes=tf.io.serialize_tensor(label2).numpy()

    feature={
        "image":tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes])),
        "label1":tf.train.Feature(bytes_list=tf.train.BytesList(value=[label1_bytes])),
        "label2":tf.train.Feature(bytes_list=tf.train.BytesList(value=[label2_bytes]))
    }

    record=tf.train.Example(features=tf.train.Features(feature=feature))
    return record
        
    
    



    
        
        

In [28]:
filepaths=get_filepaths(new_train_data_dir_path)

In [29]:
tf_record_path=os.path.join(".","Data","train_data.tfrecord")

In [30]:
tf_record_path

'./Data/train_data.tfrecord'

In [31]:
with tf.io.TFRecordWriter(tf_record_path) as writer:
    for ind,path in enumerate(filepaths):
        tf_example=preprocess_image(path)
        writer.write(tf_example.SerializeToString())
        
        if (ind+1)%1000==0:
            print(f"{ind+1} Records written...")

2025-02-23 04:06:37.135905: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


1000 Records written...
2000 Records written...
3000 Records written...
4000 Records written...
5000 Records written...
6000 Records written...
7000 Records written...
8000 Records written...
9000 Records written...
10000 Records written...
11000 Records written...
12000 Records written...
13000 Records written...
14000 Records written...
15000 Records written...
16000 Records written...
17000 Records written...
18000 Records written...
19000 Records written...
20000 Records written...
21000 Records written...
22000 Records written...
23000 Records written...
24000 Records written...
25000 Records written...
26000 Records written...
27000 Records written...
28000 Records written...
29000 Records written...
30000 Records written...
31000 Records written...
32000 Records written...
33000 Records written...
34000 Records written...
35000 Records written...
36000 Records written...
37000 Records written...
38000 Records written...
39000 Records written...
40000 Records written...
41000 Rec

In [32]:
def parse_tf_record(example):
    feature_description={
        "image":tf.io.FixedLenFeature([],tf.string),
        "label1":tf.io.FixedLenFeature([],tf.string),
        "label2":tf.io.FixedLenFeature([],tf.string)
    }
    
    example=tf.io.parse_single_example(example,feature_description)
    
    image=tf.io.parse_tensor(example["image"],out_type=tf.float32)
    label1=tf.io.parse_tensor(example["label1"],out_type=tf.float32)
    label2=tf.io.parse_tensor(example["label2"],out_type=tf.float32)
    
    return image,(label1,label2)



In [33]:
dataset=tf.data.TFRecordDataset(tf_record_path)

In [34]:
dataset=dataset.map(parse_tf_record)

In [35]:
batch_size=10

In [36]:
dataset=dataset.shuffle(buffer_size=1024).batch(batch_size)

In [37]:
for img,label in dataset:
    img=img[0]
    label1=label[0][0]
    label2=label[1][0]
    print(f"IMg:{img}")
    print(f"label1:{label1}")
    print(f"label2:{label2}")
    print(img.shape,label1.shape,label2.shape)
    break

2025-02-23 04:09:34.631302: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144


IMg:[[[0.38431373 0.38431373 0.4       ]
  [0.38431373 0.38431373 0.4       ]
  [0.38431373 0.38431373 0.4       ]
  ...
  [0.74509805 0.7294118  0.7411765 ]
  [0.74509805 0.7294118  0.7411765 ]
  [0.74509805 0.7294118  0.7411765 ]]

 [[0.38431373 0.38431373 0.4       ]
  [0.38431373 0.38431373 0.4       ]
  [0.38431373 0.38431373 0.4       ]
  ...
  [0.74509805 0.7294118  0.7411765 ]
  [0.74509805 0.7294118  0.7411765 ]
  [0.74509805 0.7294118  0.7411765 ]]

 [[0.38431373 0.38431373 0.4       ]
  [0.38431373 0.38431373 0.4       ]
  [0.38431373 0.38431373 0.4       ]
  ...
  [0.74509805 0.7294118  0.7411765 ]
  [0.74509805 0.7294118  0.7411765 ]
  [0.74509805 0.7294118  0.7411765 ]]

 ...

 [[0.3137255  0.31764707 0.29803923]
  [0.3137255  0.31764707 0.29803923]
  [0.3137255  0.31764707 0.29803923]
  ...
  [0.49803922 0.5019608  0.4862745 ]
  [0.49803922 0.5019608  0.4862745 ]
  [0.49803922 0.5019608  0.4862745 ]]

 [[0.3137255  0.31764707 0.29803923]
  [0.3137255  0.31764707 0.298039